# Pre-Processing 

**Before Running this notebook, navigate to https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud.
Download the data file and upload it into this directory (cmse492_project\Data\Pre-Processing) as well as the Raw directory (cmse492_project\Data\Raw)**

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from datetime import timedelta
import numpy as np
from sklearn.preprocessing import StandardScaler

**I. Checking for NA Values**

In [2]:
raw = pd.read_csv('creditcard.csv')
raw.head()
print(raw.shape)

(284807, 31)


In [3]:
print(raw.isnull().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [11]:
raw.dropna(axis=1)
raw.shape

(284807, 31)

**II. Feature Engineering: Creating Columns for Transaction Time and Hour of the Day**

In [12]:
df = raw.copy()
first = df['Time'][0] #first time in dataframe 
print(first)
last = df['Time'][259430] #last time in dataframe 
print(last)

0.0
159112.0


In [13]:
start_date = pd.to_datetime("2023-01-01") #create start date to work from 

df['TransactionTime'] = start_date + pd.to_timedelta(df['Time'], unit='s') #create transaction time based on time column 
df['HourOfDay'] = df['TransactionTime'].dt.hour 
df = df[['Time', 'TransactionTime', 'HourOfDay'] + [f'V{i}' for i in range(1, 29)] + ['Amount', 'Class']] #add columns

In [19]:
df.head()

,Time,TransactionTime,HourOfDay,V1,V2,V3,V4,V5,V6,V7,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,00:00:00,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,00:00:00,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,00:00:01,0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,00:00:01,0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,00:00:02,0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


**III. Checking For Highly Correlated Features**

In [23]:
corr_matrix = df.drop(columns=['Time', 'TransactionTime']).corr().abs()
corr_matrix

,HourOfDay,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
HourOfDay,1.000000,5.214205e-03,7.802199e-03,2.156874e-02,3.506295e-02,3.513442e-02,1.894502e-02,9.729167e-03,3.210647e-02,1.898298e-01,...,1.191466e-02,1.660982e-02,6.004232e-03,4.328237e-03,3.497363e-03,1.146125e-03,8.676362e-03,7.492140e-03,0.006667,0.017109
V1,0.005214,1.000000e+00,4.135835e-16,1.227819e-15,9.215150e-16,1.812612e-17,6.506567e-16,1.005191e-15,2.433822e-16,1.513678e-16,...,2.457409e-16,4.290944e-16,6.168652e-16,4.425156e-17,9.605737e-16,1.581290e-17,1.198124e-16,2.083082e-15,0.227709,0.101347
V2,0.007802,4.135835e-16,1.000000e+00,3.243764e-16,1.121065e-15,5.157519e-16,2.787346e-16,2.055934e-16,5.377041e-17,1.978488e-17,...,8.480447e-17,1.526333e-16,1.634231e-16,1.247925e-17,4.478846e-16,2.057310e-16,4.966953e-16,5.093836e-16,0.531409,0.091289
V3,0.021569,1.227819e-15,3.243764e-16,1.000000e+00,4.711293e-16,6.539009e-17,1.627627e-15,4.895305e-16,1.268779e-15,5.568367e-16,...,5.706192e-17,1.133902e-15,4.983035e-16,2.686834e-19,1.104734e-15,1.238062e-16,1.045747e-15,9.775546e-16,0.210880,0.192961
V4,0.035063,9.215150e-16,1.121065e-15,4.711293e-16,1.000000e+00,1.719944e-15,7.491959e-16,4.104503e-16,5.697192e-16,6.923247e-16,...,1.949553e-16,6.276051e-17,9.164206e-17,1.584638e-16,6.070716e-16,4.247268e-16,3.977061e-17,2.761403e-18,0.098732,0.133447
V5,0.035134,1.812612e-17,5.157519e-16,6.539009e-17,1.719944e-15,1.000000e+00,2.408382e-16,2.715541e-16,7.437229e-16,7.391702e-16,...,3.920976e-16,1.253751e-16,8.428683e-18,1.149255e-15,4.808532e-16,4.319541e-16,6.590482e-16,5.613951e-18,0.386356,0.094974
V6,0.018945,6.506567e-16,2.787346e-16,1.627627e-15,7.491959e-16,2.408382e-16,1.000000e+00,1.191668e-16,1.104219e-16,4.131207e-16,...,5.833316e-17,4.705235e-19,1.046712e-16,1.071589e-15,4.562861e-16,1.357067e-16,4.452461e-16,2.594754e-16,0.215981,0.043643
V7,0.009729,1.005191e-15,2.055934e-16,4.895305e-16,4.104503e-16,2.715541e-16,1.191668e-16,1.000000e+00,3.344412e-16,1.122501e-15,...,2.027779e-16,8.898922e-16,4.387401e-16,7.434913e-18,3.094082e-16,9.657637e-16,1.782106e-15,2.776530e-16,0.397311,0.187257
V8,0.032106,2.433822e-16,5.377041e-17,1.268779e-15,5.697192e-16,7.437229e-16,1.104219e-16,3.344412e-16,1.000000e+00,4.356078e-16,...,3.892798e-16,2.026927e-16,6.377260e-17,1.047097e-16,4.653279e-16,1.727276e-16,1.299943e-16,6.200930e-16,0.103079,0.019875
V9,0.189830,1.513678e-16,1.978488e-17,5.568367e-16,6.923247e-16,7.391702e-16,4.131207e-16,1.122501e-15,4.356078e-16,1.000000e+00,...,1.936953e-16,7.071869e-16,5.214137e-16,1.430343e-16,6.757763e-16,7.888853e-16,6.709655e-17,1.110541e-15,0.044246,0.097733


In [24]:
correlation_threshold = 0.85
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
print(to_drop)

[]


In [25]:
lower = corr_matrix.where(np.tril(np.ones(corr_matrix.shape), k=-1).astype(bool))
to_drop = [column for column in lower.columns if any(lower[column] > correlation_threshold)]
print(to_drop)

[]


There exists no overly correlated (>.85) features

**IV: Importing New Data to CSV**

In [26]:
df.to_csv('processed.csv', index=False)